# Python for Bionformatics

## Datasets

[Click here](Datasets.py) for the datasets used in the book. You only need this if you do not use the notebooks (as the notebooks will take care of the data)


## Python and the surrounding software ecology

- [Interfacing with R](Chapter01/Interfacing_R.py)
- [R Magic](Chapter01/R_magic.py)

In [35]:
#
email_address = "kecinjspring@gmail.com"

In [1]:
import os

# Mount Drive and set the working directory
from google.colab import drive
drive.mount('/content/drive')

# Set working directory
working_directory = '/content/drive/MyDrive/DS_Projects/bioinformatics'
os.makedirs(working_directory, exist_ok=True)
os.chdir(working_directory) # Create the 'data' folder if it doesn't exist

# Download the data file Human 1000 Genome Project
!wget -nd http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/phase3/20130502.phase3.sequence.index -O sequence.index

Mounted at /content/drive
--2023-09-10 16:27:04--  http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/phase3/20130502.phase3.sequence.index
Resolving ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)... 193.62.193.167
Connecting to ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)|193.62.193.167|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67057948 (64M)
Saving to: ‘sequence.index’

sequence.index      100%[===================>]  63.95M  45.2MB/s    in 1.4s    

2023-09-10 16:27:06 (45.2 MB/s) - ‘sequence.index’ saved [67057948/67057948]



In [2]:
# Chapter 3: Next-Generation Sequencing

# Install biopython if not already installed
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.0 MB/s eta 0:00:00


In [3]:
# Fetch data from NCBI database
from Bio import Entrez, SeqIO
Entrez.email = 'kevinjspring@gmail.com'

In [10]:
# Find the chloroquine resistance transporter (CRT) gene in Plasmodium falciparum

handle = Entrez.esearch(db='nucleotide', term='CRT[Gene Name] AND "Plasmodium falciparum" [Organism]')

rec_list = Entrez.read(handle)

if int(rec_list['RetMax']) < int(rec_list['Count']):
  handle = Entrez.esearch(db='nucleotide',
                          term = 'CRT[Gene Name] AND "Plasmodium falciparum" [Organism]',
                          retmax=rec_list['Count'])

  rec_list = Entrez.read(handle) # Data is stored in Dictionary

# Print the total number of records and keys in the dictionary
print(rec_list['Count'])
for key in rec_list.keys():
  print(key)

3080
Count
RetMax
RetStart
IdList
TranslationSet
TranslationStack
QueryTranslation


In [21]:
# Retrieve all the records
id_list = rec_list['IdList']
hdl = Entrez.efetch(db='nucleotide', id=id_list, retype='gb')

Above is not the best strategy. One way to make a more restrictive query is to download just a few at a time and stop when you have found the one you need.

In [24]:
# Fetch all the records using efetch
hdl = Entrez.efetch(db='nucleotide', id=','.join(id_list), rettype='gb', retmode='text')

# Parse the records into a list
recs = list(SeqIO.parse(hdl, 'gb'))

# Find the record with the name 'KM288867'
for rec in recs:
    if rec.name == 'KM288867':
        break

print(rec.name)
print(rec.description)

KM288867
Plasmodium falciparum clone PF3D7_0709000 chloroquine resistance transporter (CRT) gene, complete cds


In [25]:
# Fetch only the 'KM288867' record using efetch
hdl = Entrez.efetch(db='nucleotide', id='KM288867', rettype='gb', retmode='text')

# Parse the record
rec = SeqIO.read(hdl, 'gb')

print(rec.name)
print(rec.description)


KM288867
Plasmodium falciparum clone PF3D7_0709000 chloroquine resistance transporter (CRT) gene, complete cds


In [26]:
# extract sequence features that contain information such as gene products and exon positions
for feature in rec.features:
  if feature.type == 'gene':
    print(feature.qualifiers['gene'])
  elif feature.type == 'exon':
    loc = feature.location
    print(loc.start, loc.end, loc.strand)
  else:
    print('not processed:\n%s' % feature)

not processed:
type: source
location: [0:10000](+)
qualifiers:
    Key: clone, Value: ['PF3D7_0709000']
    Key: db_xref, Value: ['taxon:5833']
    Key: mol_type, Value: ['genomic DNA']
    Key: organism, Value: ['Plasmodium falciparum']

['CRT']
not processed:
type: mRNA
location: join{[2751:3543](+), [3720:3989](+), [4168:4341](+), [4513:4646](+), [4799:4871](+), [4994:5070](+), [5166:5249](+), [5376:5427](+), [5564:5621](+), [5769:5862](+), [6055:6100](+), [6247:6302](+), [6471:7598](+)}
qualifiers:
    Key: gene, Value: ['CRT']
    Key: product, Value: ['chloroquine resistance transporter']

not processed:
type: 5'UTR
location: [2751:3452](+)
qualifiers:
    Key: gene, Value: ['CRT']

not processed:
type: primer_bind
location: [2935:2958](+)
qualifiers:

not processed:
type: primer_bind
location: [3094:3121](+)
qualifiers:

not processed:
type: CDS
location: join{[3452:3543](+), [3720:3989](+), [4168:4341](+), [4513:4646](+), [4799:4871](+), [4994:5070](+), [5166:5249](+), [5376:54

In [27]:
# Look at the annotations on the record
for name, value in rec.annotations.items():
  print('%s=%s' % (name, value))

molecule_type=DNA
topology=linear
data_file_division=INV
date=12-NOV-2014
accessions=['KM288867']
sequence_version=1
keywords=['']
source=Plasmodium falciparum (malaria parasite P. falciparum)
organism=Plasmodium falciparum
taxonomy=['Eukaryota', 'Sar', 'Alveolata', 'Apicomplexa', 'Aconoidasida', 'Haemosporida', 'Plasmodiidae', 'Plasmodium', 'Plasmodium (Laverania)']
references=[Reference(title='Versatile control of Plasmodium falciparum gene expression with an inducible protein-RNA interaction', ...), Reference(title='Direct Submission', ...)]


In [32]:
# print part of the sequence and total length
print(rec.seq[1:100])
print(len(rec.seq))

TATGTAAAACCAAAATAAATTAAACAGAATTTATTTTTAAAAGATTTATTTGTAACAATATTACCATGATGATTTATTAAAGTAAAATCACCACCTATT
10000


In [33]:
# Get reference
from Bio import Medline
refs = rec.annotations['references']
for ref in refs:
  print(ref.pubmed_id)

25370483



In [37]:
# Performing basic sequence analysis
from Bio import Entrez, SeqIO, SeqRecord
Entrez.email = email_address

# Use the human lactase (LCT) gene for this example
hdl = Entrez.efetch(db='nucleotide',
                    id = ['NM_002299'],
                    rettype='gb') # Lactase gene
gb_rec = SeqIO.read(hdl, 'gb')

In [38]:
# Get location of the gene
for feature in gb_rec.features:
  if feature.type == 'CDS':
    location = feature.location # Note translation existing
cds = SeqRecord.SeqRecord(gb_rec.seq[location.start:location.end],
                          'NM_002299',
                          description="LCT CDS only")

In [42]:
print(cds)

ID: NM_002299
Name: <unknown name>
Description: LCT CDS only
Number of features: 0
Seq('ATGGAGCTGTCTTGGCATGTAGTCTTTATTGCCCTGCTAAGTTTTTCATGCTGG...TGA')


In [47]:
# Save file disk
with open('example.fasta', 'w') as f:
    SeqIO.write([cds], f, 'fasta')
    # file will be closed when ending this block
